In [ ]:
import sys
import os
import datetime
import numpy as np
import pandas as pd
import folium
import matplotlib.pyplot as plt

import json

cur_dir = os.getcwd()
root_dir = os.path.dirname(cur_dir+"/../../")
sys.path.append(root_dir)

image_dir = cur_dir + "/output/"
data_dir = cur_dir + "/data/"
data_path = data_dir + '/airports.csv'

# Reddit

Loading reddit api keys

In [ ]:
from dotenv import load_dotenv

# Load environment variables from a .env file
dotenv_path = cur_dir + '/reddit-api.env'
load_dotenv(dotenv_path=dotenv_path)

# Access your API key
client_id = os.getenv('client_id')
client_secret = os.getenv('client_secret')

In [16]:
import praw
from transformers import BertTokenizer, BertForTokenClassification, pipeline

# Set up PRAW with your Reddit app credentials
reddit = praw.Reddit(client_id='osjO-OqSVLhQYCvv95WhLw',
                     client_secret='7Qs6BM9criBAizA73dmwNLBi1EZ3Ig',
                     user_agent=True)

def scrape_reddit(city):
    subreddit = reddit.subreddit(city)
    query = "attractions"
    posts = subreddit.search(query, limit=100)
    
    post_titles = [post.title for post in posts]
    return post_titles

def extract_place_names(texts):
    model_name = "dbmdz/bert-large-cased-finetuned-conll03-english"
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForTokenClassification.from_pretrained(model_name)

    nlp = pipeline("ner", model=model, tokenizer=tokenizer)

    place_names = []
    for text in texts:
        ner_results = nlp(text)
        place_name = ' '.join([result['word'] for result in ner_results if result['entity'] == 'I-LOC'])
        place_names.append(place_name)
    return place_names

city = "longbeach"
reddit_posts = scrape_reddit(city)
extracted_place_names = extract_place_names(reddit_posts)
print(extracted_place_names)

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['', '', '', 'Queen Mary California', 'Or ##iza ##ba', 'LA', '', '', '', '', '', '', 'Long Beach', 'A ##qua ##rium of the Pacific', '', 'Seal Beach', '', 'Long Beach', '', '', 'Long Beach Pike', 'Hard Rock Hotel', '', '', '', '', '', 'Long Beach LA Atlantic', '', '', '', '', 'Long Beach Fuller ##ton', 'Bell ##flower B ##l ##d', 'Long Beach', 'Queen Mary', '', '', '', '', 'Beach', '', '', 'Long Beach', '', 'Long Beach', 'Long Beach', '', '', '', 'San Pedro', '', '', 'Hawaiian Gardens', 'Out ##let Mall The Pike Rainbow Harbor', '', 'Long Beach', 'Art Theatre', '', 'Downtown The Harbor', '', 'Al ##ami ##tos Bay', '', 'CA', '', 'Dark Harbor']


In [17]:
from collections import Counter

word_counts = Counter(extracted_place_names)
for word, count in word_counts.items():
    print(f"{word}: {count} occurrences")

: 39 occurrences
Queen Mary California: 1 occurrences
Or ##iza ##ba: 1 occurrences
LA: 1 occurrences
Long Beach: 7 occurrences
A ##qua ##rium of the Pacific: 1 occurrences
Seal Beach: 1 occurrences
Long Beach Pike: 1 occurrences
Hard Rock Hotel: 1 occurrences
Long Beach LA Atlantic: 1 occurrences
Long Beach Fuller ##ton: 1 occurrences
Bell ##flower B ##l ##d: 1 occurrences
Queen Mary: 1 occurrences
Beach: 1 occurrences
San Pedro: 1 occurrences
Hawaiian Gardens: 1 occurrences
Out ##let Mall The Pike Rainbow Harbor: 1 occurrences
Art Theatre: 1 occurrences
Downtown The Harbor: 1 occurrences
Al ##ami ##tos Bay: 1 occurrences
CA: 1 occurrences
Dark Harbor: 1 occurrences


In [11]:
reddit_posts

['Shoutout to the tall black guy on Shoreline today that helped me and my traveling partner get two Japanese tourists their stuff back from a crackhead.',
 'Best Place For Tourists To Eat',
 'What’s your favorite place to travel just to get away for a few days.',
 'Best place for PCR test for international travel?',
 'In case your family is too far away to travel this Thanksgiving, we have food and a place at the table for you',
 "If a friend was visiting you for the first time, where are some places you'd take them?",
 'The traveling piglet needs to get to Los Angeles and is looking for a travel companion from New York or Minneapolis.',
 'Affordable PCR covid-19 testing for International Travel',
 'Health Order Update (12/15/21) - Updates for mega events, workplace mask guidance, travel recommendations and youth sports protocols',
 'Aussie Tourist']

In [3]:
# Load the pre-trained model and tokenizer
from transformers import AutoModelForTokenClassification, AutoTokenizer
from transformers import pipeline
from collections import Counter

model_name = "dbmdz/bert-large-cased-finetuned-conll03-english"
model = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define the NER pipeline
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

# Combine the text data from different sources
text_data = wiki_text # + " ".join(articles)

# Apply the NER model to the text data
ner_results = nlp(text_data)

# Extract location entities
location_entities = [result['word'] for result in ner_results if result['entity'] == 'I-LOC']

# Count the occurrences of each location
location_counts = Counter(location_entities)

# Display the most popular locations
most_popular_locations = location_counts.most_common()

print("Most popular locations in the given city:")
for location, count in most_popular_locations:
    print(f"{location}: {count} mentions")

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Most popular locations in the given city:
Paris: 3 mentions
France: 1 mentions
City: 1 mentions
of: 1 mentions
Light: 1 mentions


# Wikipedia

In [ ]:
import requests

def get_wikipedia_page(city):
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{city}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json().get('extract', '')
    return ''

# Example usage
city = 'Long Beach'
wiki_text = get_wikipedia_page(city)

print(f"Extracted Wikipedia content for {city}: {wiki_text[:500]}...")

In [ ]:
import wikipediaapi
import wikipedia

def get_full_wikipedia_page(city):
 
    page = wikipedia.page(city).content

    if len(page) > 0:
        return page
    else:
        return ''

# Example usage
city = 'Long Beach'
wiki_text = get_full_wikipedia_page(city)

print(f"Extracted full Wikipedia content for {city} (first 500 characters):\n{wiki_text[:500]}...")

In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
from transformers import pipeline

# Load the pre-trained model and tokenizer
model_name = "dbmdz/bert-large-cased-finetuned-conll03-english"
model = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define the NER pipeline
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

# Apply the NER model to the Wikipedia text
ner_results = nlp(wiki_text)

# Extract location entities
location_entities = [result['word'] for result in ner_results if result['entity'] in ['B-LOC', 'I-LOC']]

# Remove duplicates and filter for unique attractions
unique_locations = list(set(location_entities))

print("Extracted tourist attractions:")
for location in unique_locations:
    print(location)

# Tripadvisor

In [4]:
import requests
from bs4 import BeautifulSoup

def scrape_tripadvisor(city):
    
    url = f"https://www.tripadvisor.com/Attractions-g32648-Activities-oa0-%s_California.html" % city
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # This is an example of extracting place names, it might need to be adjusted based on the actual HTML structure of TripAdvisor
    places = soup.find_all('div', class_='result-title')
    place_names = [place.get_text().strip() for place in places]
    return place_names

city = "Long_Beach"
places = scrape_tripadvisor(city)
print(places)


[]


In [3]:
import requests
url = 'https://www.reddit.com/r/longbeach/comments/1e7cd2g/local_food_pantries_in_long_beach/'

requests.get(url)

<Response [200]>

In [30]:
req = urllib.request.Request('https://www.tripadvisor.com/Attractions-g32648-Activities-oa0-Long_Beach_California.html')
req.add_headers('User-Agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:106.0) Gecko/20100101 Firefox/106.0')
req.add_header('Accept', 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8')
req.add_header('Accept-Language', 'en-US,en;q=0.5')

r = urllib.request.urlopen(req).read().decode('utf-8')
with open("test.html", 'w', encoding="utf-8") as f:
    f.write(r)

AttributeError: 'Request' object has no attribute 'add_headers'

In [3]:
from bs4 import BeautifulSoup
import requests

url = "https://www.tripadvisor.com/Attraction_Review-g319796-d5988326-Reviews-or50-Museo_de_Altamira-Santillana_del_Mar_Cantabria.html"
headers = {'User-Agent': 'Mozilla/5.0'}

r = requests.get(url,headers=headers)
data = r.text
soup = BeautifulSoup(data)

In [17]:
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html')

In [19]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en-US">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots"/>
  <!-- Hubbub v.2.20.0 https://morehubbub.com/ -->
  <meta content="en_US" property="og:locale"/>
  <meta content="article" property="og:type"/>
  <meta content="Ultimate Guide To Long Beach, California | We Are Travel Girls" property="og:title"/>
  <meta content="Ultimate guide to visiting Long Beach, with the 10 best things to do in the city, where to stay, where to eat and drink and a 3 day itinerary." property="og:description"/>
  <meta content="https://wearetravelgirls.com/long-beach-california/" property="og:url"/>
  <meta content="We Are Travel Girls" property="og:site_name"/>
  <meta content="2024-03-14T21:51:39+00:00" property="og:updated_time"/>
  <meta content="2021-06-01T06:00:36+00:00" property="article:published_time"/>
  <

In [1]:
import requests

url = "https://api.content.tripadvisor.com/api/v1/location/616578/details?key=469BD4FFBE354C519D99950F00DF2CB3&language=en&currency=USD"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{
  "error": {
    "message": "invalid client key: 469BD4FFBE354C519D99950F00DF2CB3",
    "type": "UnauthorizedException",
    "code": "160"
  }
}


In [2]:
import requests
from bs4 import BeautifulSoup

#url = 'https://www.tripadvisor.com/Tourism-g30196-Austin_Texas-Vacations.html'

url = 'https://www.tripadvisor.com/Attractions-g30196-Activities-c57-Austin_Texas.html'

response = requests.get(url, headers={'User-Agent': "Mozilla/5.0"})

soup = BeautifulSoup(response.text, 'html.parser')

items = soup.find_all('span', {'name': 'title'})

for i in items:
    print(i.text)